# Instagram scraper

## CREDITS

In [ ]:
import requests
import json

creds = dict() # dictionary to hold everything
#creds['access_token'] = 'EAAmFZATZCLgxgBAGUYzNF08WPhKdW2c0ZAhM4p2qyo8VdZCf2hmXoTU424DyVmsfB7lPgQbBeAhGpWrUgsDRSrUhkITXf3qb0fCljjPwy1Eug41n3cCuvfhni26qV9ZBqHLZA4daz4A85G40OWcmgRmFZC3npHF2r5RIE0ijN1ZCHQZDZD' # access token for use with all api calls
#creds['client_id'] = '2679944698888984' # client id from facebook app IG Graph API Test
#creds['client_secret'] = '390ee574a4874770353dc3ad66ed988c' # client secret from facebook app
creds['graph_domain'] = 'https://graph.facebook.com/' # base domain for api calls
creds['graph_version'] = 'v7.0' # version of the api we are hitting
creds['endpoint_base'] = creds['graph_domain'] + creds['graph_version'] + '/' # base endpoint with domain and version
creds['debug'] = 'no' # debug mode for api call
creds['page_id'] = '107303807767474' # users page id
creds['instagram_account_id'] = '17841440887073232' # users instagram account id
creds['ig_username'] = 'brandselfies_business' # ig username

#backup brandselfies2
creds['access_token'] = "EAAEqg3ZAgchoBACz4xLWZCRc311NTVOB8SCAMkCYYpj0gYOLTzyAWOFDdDNjnG9fZCxxLCSBU5ObMFtrCaS0oIwyXZADOsq5Dm6LZCsz41YZACbO4xfYtSCUc21NQZBjSrqfc1G2xCeeH54lrZCnLMMn1WM9kYTTKVQZD"
creds['client_id'] = '328219091825178' # client id from facebook app IG Graph API Test
creds['client_secret'] = 'c3ec335993b83d2abbdf9d828ed76219' # client secret from facebook app



## SEARCH BY HASHTAG

### Enter Hashtag and get Hashtag ID

In [ ]:
myhashtag = "selfie"

In [ ]:
#Specify fields to scrape 
fields = "id,name"

#Specify url
url = "https://graph.facebook.com/v8.0/ig_hashtag_search?user_id=" + creds['instagram_account_id']+ "&q=" + myhashtag +"&fields=" + fields + "&access_token=" + creds['access_token']

#make request and get data 
data = requests.get( url) # make get request
json_data = json.loads( data.content ) # response data from the api
hashtag_id = json_data["data"][0]["id"]

print("The hashtag is:")
print(myhashtag)

print("The hashtag_id is:")
print(hashtag_id)


### Get data by hashtag ID

In [ ]:
import sys

#https://graph.facebook.com/v8.0/{ig-hashtag-id}/top_media?user_id={user-id}&fields={fields}
	
fields = 'id,children,caption,comment_count,like_count,media_type,media_url,permalink,timestamp' # fields to get back

url = "https://graph.facebook.com/v8.0/" + hashtag_id + '/top_media?user_id=' + creds['instagram_account_id']+ "&fields=" + fields + "&access_token=" + creds['access_token']
response = []
x = 0

num_pages = 10 #number of pages to scrape

for x in range (0,num_pages):
  try:
    print(str(x+1) + ".page is being scraped with url " + str(url))
    data = requests.get(url) # make get request
    json_data = json.loads( data.content ) # response data from the api
    response.append(json_data)
    url = json_data["paging"]["next"]
    x = x+1
  except:
    break

### Transform and save data

Get list

In [ ]:
import pandas as pd

posts = []

i=0

for i in range(0,num_pages) :
  for post in response[i]['data'] : 

    try:
      id = (post['id'])
    except:
      id = ""

    try:
      permalink = (post['permalink'])
    except:
      permalink = ""
    
    try:
      media_url = post['media_url'] 
    except: 
      media_url = ""

    try:
      caption = post['caption'] 
    except: 
      caption = ""

    try:
      media_type = post['media_type'] 
    except: 
      media_type = ""

    try:
      like_count = post['like_count'] 
    except: 
      like_count = ""

    try:
      timestamp = post['timestamp'] 
    except: 
      timestamp = ""
    
    post = [id, permalink, media_url, caption, media_type, like_count, timestamp]
    #print(post)
    posts.append(post)

  i = i + 1
  print("i is now" + str(i))
    

make dataframe

In [ ]:
posts_data_frame = pd.DataFrame(posts)
posts_data_frame.columns = ["id", "permalink", "media_url", "caption", "media_type", "like_count", "timestamp"]
print(posts_data_frame.head())

save

In [ ]:
import csv

full_path = "instagram_data/data-20-09-14-hashtag-selfie.csv"


with open(full_path, 'w', encoding='utf-8', newline='') as file:
  export = posts_data_frame.to_csv(quoting=csv.QUOTE_NONNUMERIC, index=False, encoding='utf-8')
  file.write(export)

## SEARCH BY USER PROFILE

In [ ]:
user_name_search = "beerselfie"
#user_id_search = creds['instagram_account_id'] #my own account

In [ ]:
# https://graph.facebook.com/{graph-api-version}/{ig-user-id}?fields=business_discovery.username({ig-username}){username,website,name,ig_id,id,profile_picture_url,biography,follows_count,followers_count,media_count}&access_token={access-token}

url = "https://graph.facebook.com/v8.0/" + creds['instagram_account_id'] + "?fields=business_discovery.username(" + user_name_search + "){username,website,name,ig_id,id,profile_picture_url,biography,follows_count,followers_count,media_count}&access_token=" + creds['access_token']

print(url)

data = requests.get(url) # make get request
json_data = json.loads( data.content ) # response data from the api

print("username:")
print(json_data["business_discovery"]["username"])
print("\nuser_id:")
print(json_data["business_discovery"]["ig_id"])
print("\nfollows_count:")
print(json_data["business_discovery"]["follows_count"])
print("\nfollowers_count:")
print(json_data["business_discovery"]["followers_count"])
print("\nmedia_count")
print(json_data["business_discovery"]["media_count"])

user_id_search = json_data["business_discovery"]["ig_id"]
print("\nthe user id " + str(user_id_search) + " of the user " + user_name_search + " is now stored in the variable user_id_search ")


## get data

In [ ]:
#set of new access tokens:

def getNewAccessToken(token):
  mytokens = ["EAAEqg3ZAgchoBAGxW6z5YY3MzsChoQKgpP0d8CK13ilLZA5DcuEBcyeZAneS6QNnZCTtZAZAKjlAakZChZCXak0zKkIE0qZAKkcfWHA4y7xUNsjPTNOvZCCwHeDOx1FyQdYnYZBhIEEQHE2SlP0bkaxLjm9rhFiKFYKSOStLFnvdslkCQyZCcqZAhahMzDvg5qqo27X8ZD",
              2,
              3,
              4,
              5]
    
  creds['access_token'] = mytokens[token]
  return creds['access_token']

In [ ]:
fields = 'caption,media_url,media_type,like_count,comments_count,id,timestamp,permalink' # fields for first url -> EDIT DIRECTLY IN LOOP FOR PAGES 2 AND AFTER

response = []

#starting_url_vorlage = graph.facebook.com/17841440887073232?fields=business_discovery.username(beerselfie){media{caption,media_url,media_type,like_count,comments_count,id}}&access_token=EAAEqg3ZAgchoBADqSp7acTrdtmBtLvsTzZCWoVa2MUYC8Q5B5GotW5fclR4xlQbn76MLjWqGWgxNWslZBafXPKWrM6Jj7YrIjK7B1GzBs76scZCLUVcHzMrW9H7WZBOWHXcACYLaxZAdhi9ztEuYZAG2fdOOz7tP8lZBCbfhZBhm0QZBtfBRRGsGNEIW9mPmkiOYUZD
#paging_url = "https://graph.facebook.com/v2.12/" + creds['instagram_account_id'] + "?fields=business_discovery.username(" + user_name_search +")"+ "{id,name,username,website,profile_picture_url,biography,followers_count,media_count,media.after" +"("+after+"){id,caption,comments_count,like_count,media_type,media_url,owner,timestamp}}&access_token="+creds['access_token']

starting_url = "https://graph.facebook.com/" + creds['instagram_account_id'] + "?fields=business_discovery.username(" + user_name_search +"){media{" + fields + "}}&access_token=" + creds['access_token']
url = starting_url
i = 0
max_pages = 498
token = 0

for i in range (0,max_pages):
  try: 
    print("Page" + str(i) + " with url " + url + " is being scraped")
    data = requests.get(url) # make get request
    json_data = json.loads( data.content ) # response data from the api
    response.append(json_data)
    after = json_data["business_discovery"]["media"]["paging"]["cursors"]["after"]
    url = "https://graph.facebook.com/v2.12/" + creds['instagram_account_id'] + "?fields=business_discovery.username(" + user_name_search +")"+ "{id,name,username,website,profile_picture_url,biography,followers_count,media_count,media.after" +"("+after+"){id,caption,comments_count,like_count,media_type,media_url,owner,timestamp,permalink}}&access_token="+creds['access_token']
    i = i+1
    print("after code: " + str(after))
  except:
    print("Something went wrong, trying again with new access token")
    getNewAccessToken(token)
    token = token + 1


## transform and save data

In [ ]:
import pandas as pd

posts = []

i=0

for i in range(0,max_pages) :
  for post in response[i]["business_discovery"]["media"]["data"] : 

    try:
      id = (post['id'])
    except:
      id = ""

    try:
      permalink = (post['permalink'])
    except:
      permalink = ""
    
    try:
      media_url = post['media_url'] 
    except: 
      media_url = ""

    try:
      caption = post['caption'] 
    except: 
      caption = ""

    try:
      media_type = post['media_type'] 
    except: 
      media_type = ""

    try:
      like_count = post['like_count'] 
    except: 
      like_count = ""

    try:
      timestamp = post['timestamp'] 
    except: 
      timestamp = ""
    
    post = [id, permalink, media_url, caption, media_type, like_count, timestamp]
    #print(post)
    posts.append(post)

  i = i + 1
  print("i is now" + str(i))
    

In [ ]:
import csv

path = "instagram_data"
filename = "data-20-09-14-profile-beerselfie2"
fileformat = ".csv"
nr = 0

full_path = path + "/"+ filename + str(nr) + fileformat
print(full_path)


with open(full_path, 'w', encoding='utf-8', newline='') as file:
  export = posts_data_frame.to_csv(quoting=csv.QUOTE_NONNUMERIC, index=False, encoding='utf-8')
  file.write(export)

## GET PICTURES

In [ ]:
import urllib.request

path_pic = "instagram_data/images/"


for index, row in posts_data_frame.iterrows():
  try:
    urllib.request.urlretrieve(row['media_url'], path_pic+str(row['id'])+".jpg" )
  except:
    print("post with id " + row['id'] + " could not be saved" )